In [ ]:
from typing import Tuple, Optional

import numpy as np
from matplotlib import pyplot as plt
import camb

plt.rcParams.update({"figure.figsize": (12, 8), "font.size": 14})

In [ ]:
# random number generator with a fixed seed
rng = np.random.default_rng(12345)

### Get power spectrum

In [ ]:
# cosmological parameters used to generate the power spectrum
cparams = camb.CAMBparams(ombh2=0.05 * 0.67**2, omch2=0.25 * 0.67**2, H0=67.)

# interpolation function for the matter power spectrum
pk = camb.get_matter_power_interpolator(cparams, nonlinear=False, zmax=1000)

In [ ]:
# Define a k axis to make some plots
# CAMB uses units of Mpc/h
kh = np.logspace(-4, 1, 1000)

In [ ]:
# Plot the power spectrum at a few redshifts
for z in [100, 10, 1, 0]:
    plt.plot(kh, pk.P(z, kh), label=f"z={z}")
plt.legend()
plt.loglog()
plt.xlabel("$k/h$ (h/Mpc)")
plt.ylabel("$P(k)$ $(Mpc/h)^3$")

### Draw a realisation of density contrast
We will draw random gaussian fluctuations from the power spectrum.

In [ ]:
# Define a discrete grid and spacing in Mpc/h
# We use a box with two long sides (N) and one short side (M) to save some memory
N = 512
M = 16
d = 2  # Mpc / h spacing

# Draw a Gaussian white noise realisation in real-space, the short axis is the last one.
# We will transform this into k-space, apply the power spectrum and transform back. The 
# rationale for starting in real-space is that we don't need to figure the internal
# normalisation of the FFT routines, just assume that the forward and reverse transforms
# are inverses.
white_noise = np.random.standard_normal((N, N, M))

# As we're dealing with a real-valued field, we will use a real-to-complex FFT.
# This returns one axis with only the positive wavenumbers (as the negative frequencies
# are the complex conjugates). By convention this is the last axis, which is also the
# short axis.
# Use the numpy `fftfreq` routines to get the wavenumbers for each position on the 
# long (x, y) or short (z) axes. The factor of 2 pi is because in cosmology we an angular
# frequency convention (i.e. the phase factor in the FT is e^{i k x}), whereas FFT
# routines typically use a regular frequency convention (i.e. e^{2 pi i f t}).
klong = 2 * np.pi * np.fft.fftfreq(N, d=d)
kshort = 2 * np.pi * np.fft.rfftfreq(M, d=d)

# Construct arrays for the wavenumbers along the x, y, z axes
kx = klong[:, np.newaxis, np.newaxis]
ky = klong[np.newaxis, :, np.newaxis]
kz = kshort[np.newaxis, np.newaxis, :]

# The total wavenumber for each element in k-space
k_grid = np.sqrt(kx**2 + ky**2 + kz**2)

# Initial redshift
z0 = 1000.0

# Get the power spectrum amplitude for each cell. Mapping the continuous power spectrum
# into a discrete version (easiest calculation computes the variance in each case and 
# finds the normalisation) generates a 1 / V_cell factor, where V_cell is the size of a
# grid cell, i.e. d^3
P_discrete = pk.P(z0, k_grid) / (d**3)

# Scale the white noise realisation in k-space, remove the constant mode (which may be ill defined), and transform back
delta_k = np.fft.rfftn(white_noise) * P_discrete**0.5
delta_k[0, 0, 0] = 0.0
delta = np.fft.irfftn(delta_k)

In [ ]:
# quick plot of k and delta^2(k)
fig, ax = plt.subplots(1, 2, figsize=(17, 6))
im = ax[0].imshow(np.fft.fftshift(k_grid[..., 0]), origin="lower")
plt.colorbar(im, ax=ax[0], label="k (h/Mpc)")
im = ax[1].imshow(np.log10(np.fft.fftshift(np.abs(delta_k[..., 0])**2)), origin="lower")
plt.colorbar(im, ax=ax[1], label="$\log_{10}\delta^2(k)$")

In [ ]:
# Plot the density contrast field at Z=0
vlim = 4 * delta.std()
fig, axis = plt.subplots(1, 1, figsize=(10, 8))

im = axis.imshow(delta[..., 0], cmap="coolwarm", origin="lower", extent=(0, N * d, 0, N * d), vmin=-vlim, vmax=vlim)
fig.colorbar(im, label="$\delta$")
axis.set_xlabel("x (Mpc/h)")
axis.set_ylabel("y (Mpc/h)")
axis.set_title("z = 0 Mpc/h")

# zoom in if desired
#axis.set_xlim(-100, 100)
#axis.set_ylim(-100, 100)

### Estimate Power Spectrum from Realisation

In [ ]:
def estimate_ps(delta: np.ndarray, d: float, N: int = 100, kmax: Optional[float] = None, XY: bool = False):
    """Estimate the power spectrum.
    
    Parameters
    ----------
    delta
        Density field to take the power spectrum of.
    d
        Size of grid cell.
    N
        Number of output power spectrum bins.
    kmax
        Max k to go up to. If not set, use the highest present k.
    XY
        Estimate the power spectrum using the XY dimensions only. This is useful if the Z 
        dimension is much smaller which can cause artifacts.
        
    Returns
    -------
    k
        The centre of each k-bin.
    ps
        The power spectrum value.
    """
    
    # Construct arrays for the wavenumbers along the x, y, z axes
    kx = 2 * np.pi * np.fft.fftfreq(delta.shape[0], d=d)
    ky = 2 * np.pi * np.fft.fftfreq(delta.shape[1], d=d)
    kz = 2 * np.pi * np.fft.rfftfreq(delta.shape[2], d=d)
    
    k = (
        kx[:, np.newaxis, np.newaxis]**2 + 
        ky[np.newaxis, :, np.newaxis]**2 + 
        kz[np.newaxis, np.newaxis, :]**2
    )**0.5
    
    # This sets the k of each cell to be less than zero, which means it
    # will be missed from the subsequent sums.
    if XY:
        k[..., 1:] = -1.0
    
    # Define bins in k-space, and use np.digitize to find which k-space cells line in which bins
    if kmax is None:
        kmax = k.max()
    k_bins = np.linspace(0, kmax, N + 1)
    bin_ind = np.digitize(k.ravel(), k_bins)
    
    # Average power along the k bin rings using np.bincount to do a weighted count of the cells.
    delta_k = np.fft.rfftn(delta)
    pk = np.bincount(bin_ind.ravel(), weights=np.abs(delta_k.ravel())**2)
    pk_norm = np.bincount(bin_ind.ravel())
    pk = pk[1:-1] / pk_norm[1:-1] / delta.size * d**3
    
    # Construct the central value of each bin
    kc = 0.5 * (k_bins[1:] + k_bins[:-1])
    
    return kc, pk

In [ ]:
# Get power spectrum estimate from our simulated realisation
k_sim, ps_sim = estimate_ps(delta, d)

In [ ]:
plt.plot(k_sim, ps_sim, label="simulation")
plt.plot(k_sim, pk.P(z0, k_sim), label="true")
plt.loglog()
plt.legend()
plt.ylim(1e-5, 1e-1)
plt.xlabel("k (h/Mpc)")

### Calculate Zel'dovich Power Spectrum

In [ ]:
def grid_particles(
    particles: np.ndarray,
    N: Tuple[float, float, float],
    L: Tuple[float, float, float],
    W: Tuple[float, float, float],
) -> np.ndarray:
    """Assign the particle mass onto a 3D grid
    
    Particles are periodically wrapped into the grid.
    
    Parameters
    ----------
    particles
        A two dimensional array of particle positions. The array must be
        shaped as (3, num_particles).
    N
        The size of the output box along the X, Y and Z dimensions.
    L
        The center of the lowest cell in each dimensions.
    W
        The width of the box in each dimension.
        
    Returns
    -------
    density_grid
        The mass of the particles
    """
    import pynbody
    
    particles = particles.copy()
    
    # Wrap the particles over the periodic boundary to whichever side will be nearest
    for ii, (n, s, w) in enumerate(zip(N, L, W)):
        d = w / n
        particles[ii] = ((particles[ii] - s + 0.5 * d) % w) - 0.5 * d + s
    
    # Construct a pynbody snapshot
    nb = pynbody.new(particles.shape[1])
    nb["x"] = particles[0].ravel()
    nb["y"] = particles[1].ravel()
    nb["z"] = particles[2].ravel()
    nb["mass"] = 1.0
    
    # Place on a grid, and construct the fractional density contrast
    gr = pynbody.sph.to_3d_grid(nb, nx=N[0], ny=N[1], nz=N[2])
    gr /= gr.mean()
    gr -= 1.0
    
    return gr